In [ ]:
import cv2
import numpy as np
import imutils
import os
from pathlib import Path
from scipy import ndimage as ndi
from skimage import io
from matplotlib import pyplot as plt
from keras.preprocessing import image
from google.colab import files, drive
drive.mount("/content/drive", force_remount=False)

In [ ]:
img = io.imread("drive/MyDrive/CSC381/Images/my_coins2/coins3.jpg")
redc = img[:,:,0]
kernel = np.ones((3,3),np.uint8)

def hue_Thresh(image):
  # used with my_coins2, thresholds coins from red background image
  # returns binary image of coins 
  hsv_img = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
  redc = img[:,:,0]
  red_mask = cv2.inRange(hsv_img, (115, 100, 20), (130, 255, 255))
  image1 = cv2.bitwise_and(redc, redc, mask=red_mask)
  ret, thresh1 = cv2.threshold(image1, 120, 255, cv2.THRESH_BINARY)
  thresh1 = cv2.bitwise_not(thresh1)
  return thresh1

plt.imshow(hue_Thresh(img), cmap="binary_r")

In [ ]:
# code segment for my_coins2
path_to_image_folder = "drive/MyDrive/CSC381/Images/my_coins2"
paths = Path(path_to_image_folder).glob('**/*.jpg')
thresh_imgs = []
quarts = []
nicks = []
dimes = []
pens = []

img_num = 0
for path in paths:
  img_num += 1 
  circles = []
  img = io.imread(path)
  gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  redc = img[:,:,0]
  kernel = np.ones((3,3),np.uint8)

  thresh1 = hue_Thresh(img)

  thresh_imgs.append(thresh1)

  output = img.copy()
  # find contours in the thresholded image
  cnts = cv2.findContours(thresh1.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)
  cnts = imutils.grab_contours(cnts)
  # loop over the contours, grab data
  text = ""
  sum = 0
  for (i, c) in enumerate(cnts):
    # find contour area
    area = cv2.contourArea(c)
    # pick contours large enough to surround coin 
    if area > 50000:
        ((x, y), _) = cv2.minEnclosingCircle(c)
        if area >= 550000: # quarter
          sum += 25
          quarts.append(area)
          text = "Q"
        elif area >= 385000: # nickel
          sum += 5
          nicks.append(area)
          text = "N"
        elif area >= 335000: # penny
          sum += 1
          pens.append(area)
          text = "P"
        elif area >= 280000: # dime
          sum += 10
          dimes.append(area)
          text = "D"
        circles.append((area, text))
        cv2.putText(output, "{}".format(text), (int(x) - 10, int(y)),
        cv2.FONT_HERSHEY_SIMPLEX, 5, (255, 0, 0), 8)
        cv2.drawContours(output, [c], -1, (0, 255, 0), 2)

  cv2.putText(output, "File = {}".format(path), (100, 100),
      cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 0, 255), 8)
  cv2.putText(output, "Sum = {}".format(sum), (100, 300),
      cv2.FONT_HERSHEY_SIMPLEX, 6, (0, 0, 255), 8)
  
  print("[INFO] File = {}".format(path))
  print("[INFO] Sum = {}.".format(sum))
  print("[INFO] Coins = {}".format(circles))
  io.imshow(output)
  io.imsave("output{}.jpg".format(img_num),output)

In [ ]:
# best thresholds set for my_coins1 ONLY

path_to_image_folder = "drive/MyDrive/CSC381/Images/my_coins"
paths = Path(path_to_image_folder).glob('**/*.jpg')
threshes = []
img_num = 0
for path in paths:
  img_num += 1 

  img = io.imread(path)
  img = img.astype(np.uint8)
  gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

  kernel = np.ones((7,7),np.uint8)
  #gray = cv2.medianBlur(gray, 15)
  thresh = cv2.adaptiveThreshold(gray,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,115,15)
  thresh = cv2.bitwise_not(thresh)
  thresh = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel, iterations = 2)
  thresh = ndi.binary_fill_holes(thresh)

  #outputcoins9 (try smaller dilation? or )
  thresh = cv2.dilate(thresh.astype(np.uint8), kernel, iterations = 1)
  threshes.append(thresh)
  output = img.copy()
  # find contours in the thresholded image
  thresh = thresh.astype(np.uint8)
  cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL,
    cv2.CHAIN_APPROX_SIMPLE)
  cnts = imutils.grab_contours(cnts)
  circles = []
  # loop over the contours, grab data
  text = ""
  sum = 0
  quarts = 0
  nicks = 0
  dimes = 0
  pens = 0
  for (i, c) in enumerate(cnts):
    # find contour area
    area = cv2.contourArea(c)
    # pick contours large enough to surround coin 
    if area > 150000:
        ((x, y), _) = cv2.minEnclosingCircle(c)
        if area >= 430000: # quarter
          sum += 25
          quarts += 1
          text = "{}".format("Q")
        elif area >= 350000: # nickel
          sum += 5
          nicks += 1
          text = "{}".format("N")
        elif area >= 272000: # penny
          sum += 1
          pens += 1
          text = "{}".format("P")
        elif area >= 180000: # dime
          sum += 10
          dimes += 1
          text = "{}".format("D")
        circles.append((area, text))
        cv2.putText(output, "{}".format(text), (int(x) - 10, int(y)),
        cv2.FONT_HERSHEY_SIMPLEX, 5, (0, 255, 255), 8)
        cv2.drawContours(output, [c], -1, (0, 255, 0), 6)

  cv2.putText(output, "File = {}".format(path), (100, 100),
      cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 0, 255), 8)
  cv2.putText(output, "Sum = {}".format(sum), (100, 300),
      cv2.FONT_HERSHEY_SIMPLEX, 6, (0, 0, 255), 8)
  
  print("[INFO] File = {}".format(path))
  print("[INFO] Sum = {}.".format(sum))
  print("[INFO] Coins = {}".format(circles))
  io.imshow(output)
  io.imsave("output{}.jpg".format(img_num),output)


In [ ]:
def count_quarters(thresh, output, sum):
  text = ""
  quarts = []
  nicks = []
  dimes = []
  pens = []  
  minRadius = 400
  maxRadius = 460
  circles = cv2.HoughCircles(thresh.astype("uint8"),cv2.HOUGH_GRADIENT,10,minRadius/2,param1=1,param2=740,minRadius=minRadius,maxRadius=maxRadius)
  if circles is not None:
    # convert to integers
    circles = np.round(circles[0, :]).astype("int")
    # loop over circles
    for (x, y, r) in circles:
      # draw circles, rectangles
      cv2.circle(output, (x, y), r, (0, 255, 0), 8)
      cv2.rectangle(output, (x - 5, y - 5), (x + 5, y + 5), (0, 128, 255), -1)
      if r >= 200:
        pens.append(area)
        sum += 25
        text = "Q"
      cv2.putText(output, "{}".format(text), (int(x) - 10, int(y)),cv2.FONT_HERSHEY_SIMPLEX, 6, (255, 0, 0), 8)
  return sum

In [ ]:
def count_nickels(thresh, output, sum):
  text = ""
  quarts = []
  nicks = []
  dimes = []
  pens = []  
  minRadius = 360
  maxRadius = 400
  circles = cv2.HoughCircles(thresh.astype("uint8"),cv2.HOUGH_GRADIENT,10,minRadius/2,param1=1,param2=850,minRadius=minRadius,maxRadius=maxRadius)
  if circles is not None:
    # convert to integers
    circles = np.round(circles[0, :]).astype("int")
    # loop over circles
    for (x, y, r) in circles:
      # draw circles, rectangles
      cv2.circle(output, (x, y), r, (0, 255, 0), 8)
      cv2.rectangle(output, (x - 5, y - 5), (x + 5, y + 5), (0, 128, 255), -1)
      if r >= 200:
        pens.append(area)
        sum += 5
        text = "N"
      cv2.putText(output, "{}".format(text), (int(x) - 10, int(y)),cv2.FONT_HERSHEY_SIMPLEX, 6, (255, 0, 0), 8)
  return sum

In [ ]:
def count_pennies(thresh, output, sum):
  text = ""
  quarts = []
  nicks = []
  dimes = []
  pens = []
  minRadius = 330
  maxRadius = 355
  circles = cv2.HoughCircles(thresh.astype("uint8"),cv2.HOUGH_GRADIENT,10,minRadius/2,param1=1,param2=750,minRadius=minRadius,maxRadius=maxRadius)
  if circles is not None:
    # convert to integers
    circles = np.round(circles[0, :]).astype("int")
    # loop over circles
    for (x, y, r) in circles:
      # draw circles, rectangles
      cv2.circle(output, (x, y), r, (0, 255, 0), 8)
      cv2.rectangle(output, (x - 5, y - 5), (x + 5, y + 5), (0, 128, 255), -1)
      if r >= 200:
        pens.append(area)
        sum += 1
        text = "P"
      cv2.putText(output, "{}".format(text), (int(x) - 10, int(y)),cv2.FONT_HERSHEY_SIMPLEX, 6, (255, 0, 0), 8)
  return sum
   

In [ ]:
def count_dimes(thresh, output, sum):
  text = ""
  coins = []
  minRadius = 100
  maxRadius = 329
  circles = cv2.HoughCircles(thresh.astype("uint8"),cv2.HOUGH_GRADIENT,10,150,param1=1,param2=670,minRadius=minRadius,maxRadius=maxRadius)
  if circles is not None:
    # convert to integers
    circles = np.round(circles[0, :]).astype("int")
    # loop over circles
    for (x, y, r) in circles:
      # draw circles, rectangles
      cv2.circle(output, (x, y), r, (0, 255, 0), 8)
      cv2.rectangle(output, (x - 5, y - 5), (x + 5, y + 5), (0, 128, 255), -1)
      if r >= 100:
        coins.append(area)
        sum += 10
        text = "D"
      cv2.putText(output, "{}".format(text), (int(x) - 10, int(y)),cv2.FONT_HERSHEY_SIMPLEX, 6, (255, 0, 0), 8)
  return sum

In [ ]:
path_to_image_folder = "drive/MyDrive/CSC381/Images/my_coins2"
paths = Path(path_to_image_folder).glob('**/*.jpg')
img_num = 0
print(img_num)
for path in paths:
	print(path)
	img = io.imread(path)
	output = img.copy()
	sum = 0
	thresh = hue_Thresh(output)
	thresh = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations = 2)
	sum = count_quarters(thresh, output, sum)
	sum = count_nickels(thresh, output, sum)
	sum = count_pennies(thresh, output, sum)
	sum = count_dimes(thresh, output, sum)
	print(sum)
	print("[INFO] File = {}".format(path))
	print("[INFO] Sum = {}.".format(sum))
	print("[INFO] Coins = {}".format(circles))
	cv2.putText(output, "File = {}".format(path), (100, 100),
		cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 0, 255), 8)
	cv2.putText(output, "Method = {}".format(str("Hough")), (100, 150),
		cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 0, 255), 8)
	cv2.putText(output, "Sum = {}".format(sum), (100, 300),
		cv2.FONT_HERSHEY_SIMPLEX, 6, (0, 0, 255), 8)
	img_num += 1
	io.imsave("HoughOutput{}.jpg".format(img_num),output)
